# New bussines location indicator

### Coursera project capstone

This is the course captstone project for the coursera IBM machine learning specialization

### About the project

Having a good location for your business is an important factor for it's prosperity. although the hot spots in a city for a new venue are common knowledge, usually this places have a very high price given that this knowledge is correlated with higher demand. With some datamining and machine learning, some places cam be discovered as good opportunity’s that does not yet received peoples attention.

This project uses python, foursquare api, machine learning and visualization tools to give some insight on where you should open your business in a city.
It works by ranking the places in city by how hot they are for business and them finding what is the optimal place for a new business that does not collide with other similar business by location but also is on a "hot spot" in the city.

The data will on the amount of venues by location will be collected using the foursquare api for a given city. The app/notebook will be created in a generic way so that people using it can change the city name. 
A map of the city based on the amount of venues will be created and also another one for the similar venues as the person is looking to create.

A score for a given location that will be created by adding value for number of venues using mahalanobis distance as a weight and the using the opposite (decreasing score) by having similar venues.
In the and, points will be clustered together to have useful locations as a suggestion for the new business to open.

### Let's start by importing the libraries we will need

In [1]:
import pandas as pd
import numpy as np
import folium, requests, os
import folium.plugins as plugins
from bs4 import BeautifulSoup
from urllib.request import urlopen
from geopy.geocoders import Nominatim

## Defining the place of interest and search area

In [2]:
# this will be our center point in the city
lat_city_center, lng_city_center = -22.727482, -47.648811

# now, create a map with this lat and lng info
map_city = folium.Map(location=[lat_city_center, lng_city_center], zoom_start=14)
folium.CircleMarker(
        [lat_city_center, lng_city_center],
        radius=10,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.7,
        parse_html=False).add_to(map_city)  

In [3]:
# check if it is correct
map_city

In [4]:
# Defining the search grid size and location

In [5]:
number_x_points = 40
number_y_points = 40

# the farthest point of interest in the map
x_max, y_max =  -22.700566, -47.682758

# now we create a distance range in lat and lng distance measure
lat_delta = 2*np.abs(x_max - lat_city_center)
lng_delta = 2*np.abs(y_max - lng_city_center)

In [6]:
# create the matrix of points for use in the map and foursquare
lat_range = np.linspace(lat_city_center - lat_delta, lat_city_center + lat_delta, number_x_points)
lng_range = np.linspace(lng_city_center - lng_delta, lng_city_center + lng_delta, number_y_points)

In [7]:
for lat in lat_range:
    for lng in lng_range:
        folium.CircleMarker(
            [lat, lng],
            radius=10,
            color='red',
            fill=True,
            fill_color='red',
            fill_opacity=0.7,
            parse_html=False).add_to(map_city)
map_city

whe can see that we have covered most of the city with every point at around 200m from each other

## Requesting the data from the location with foursquare api

In [101]:
# Client ID and cliet secret key shoud never be stored in the notebook or other script,
# so we read it from os enviroment variable. 

CLIENT_ID = 'DVUYZ0B5CPKNIIXLYAJMGRKZBYUP1U20PUC0AKEAFQT0NDLJ' 
CLIENT_SECRET = 'VA22RFREDM2G40P4YOYWY5PRJLEQZAU5ICBYSAWH0NLN4JFB' 
VERSION = '20180605' # Foursquare API version

#CLIENT_ID = os.getenv('CLIENT_ID')          # your Foursquare ID
#CLIENT_SECRET = os.getenv('CLIENT_SECRET')  # your Foursquare Secret

Creating some help functions to get and treat the data received

In [108]:
def get_info(lat, lng):
    '''
    This function call foursquare api with given lattitude and longtude and
    returns the api response as 
    '''
    url = f'https://api.foursquare.com/v2/venues/search?&client_id={CLIENT_ID}&client_secret={CLIENT_SECRET}&v={VERSION}&ll={lat},{lng}&radius={250}&limit={500}'
    results = requests.get(url).json()
    if results['meta']['code'] == 200:
        return results['response']
    return False

In [109]:
def count_venues(response):
    '''
    This function parse the response received from the get_info() function
    and treat it to filter only the number of venues in the area and retur it's number
    '''
    try:
        if response['venues']:    
            return len(response['venues'])
    except:
        pass
    return 0

In [110]:
def count_similar(response, similar=['Café', 'Cafe', 'Coffe', 'Coffee Shops']):
    '''
    This function parse the response received from the get_info() function
    and count how many of then has similar text (eg: has the type we are looking for)
    and retur it's number
    '''
    total = 0 # we start with 0 matchs  
    try:
        for venue in response['venues']:
            for item in venue['categories']:
                if  item['pluralName'] in similar or item['shortName'] in similar or item['name'] in similar:
                    total += 1
        return total
    except:
        return 0

In [111]:
# Let's just check the functions created
r = get_info(lat_city_center, lng_city_center)
print(r)
print(f'number of coffe shops: {count_similar(r)}')
print(f'number of venues in the center of the city: {count_venues(r)}')

False
number of coffe shops: 0
number of venues in the center of the city: 0


Now, we will search in all the opoints in the map

In [96]:
full_response = []
for lat in lat_range:
    line = []
    for lng in lng_range:
        r = get_info(lat_city_center, lng_city_center)
        similar = count_similar(r)
        venues = count_venues(r)
        print(similar, venues)
        line.append((similar, venues))
    full_response.append(line)  

TypeError: 'bool' object is not subscriptable

In [94]:
len(full_response[1])

40